In [ ]:
# default_exp inventory

In [ ]:
#hide
# !pip install -q nbdev lambdasdk s3bz

In [ ]:
#hide
USER=None
PW=None

# Inventory

> upload and download inventory data from villa master backend

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from json.decoder import JSONDecodeError
from botocore.config import Config
from s3bz.s3bz import S3, Requests
from lambdasdk.lambdasdk import Lambda
from awsSchema.apigateway import Event, Response
import pandas as pd
from nicHelper.wrappers import add_class_method, add_method
from nicHelper.dictUtil import printDict
from io import BytesIO
from typing import List
import bz2, json, boto3, base64, logging, itertools , requests

In [ ]:
#export
def union(*dicts):
    return dict(itertools.chain.from_iterable(dct.items() for dct in dicts))
class Endpoints:
  '''get endpoint names from branch name'''
  def __init__(self, branchName='manual-dev'):
    self.branchName = branchName
  updateWithS3 = lambda self: f'update-inventory-s3-{self.branchName}'
  inputS3 = lambda self: f'input-bucket-{self.branchName}'
  querySingleProduct = lambda self: f'single-product-query-inventory-{self.branchName}'
  queryAll = lambda self: f'query-all-inventory-{self.branchName}'
  queryBranch = lambda self: f'query-branch-inventory-{self.branchName}'
  queryAll2 = lambda self: f'query-all-inventory2-{self.branchName}'
  
  
class InventorySdk:
  ''' interact with villa inventory database '''
  def __init__(self, branchName = 'dev', user = None, pw = None, 
               region = 'ap-southeast-1'):
    self.branchName = branchName
    self.lambdaClient = Lambda(user =user, pw=pw, region = region)
    self.user = user; self.pw = pw; self.region = region
    self.endpoint = Endpoints(branchName=branchName)
    
    
  def updateWithS3(self, data:dict, 
                   key:str = 'allProducts',
                   invocationType:str = 'Event'):
    
    # save to s3
    S3.save(key = key, 
            objectToSave = data , 
            bucket = self.endpoint.inputS3(),
            user=self.user, pw=self.pw)
    logging.info(f'saving to s3 completed')
    
    lambdaPayload = {
        'inputBucketName': self.endpoint.inputS3(),
        'inputKeyName': key
    }
    logging.info(f'input to lambda is {lambdaPayload}')
    try:
      result = self.lambdaClient.invoke(functionName= self.endpoint.updateWithS3() 
                                        ,input=lambdaPayload,
                                        invocationType= invocationType )
      if result: return Response.getReturn(result)
    except JSONDecodeError:
      logging.warning('no return from function')
      return True

  def querySingleProduct(self, ib_prcode= None, functionName=None, 
                         user=None, pw=None):
    '''query a single product'''
    functionName = functionName or self.endpoint.querySingleProduct()
    input = { "body": json.dumps({'ib_prcode': ib_prcode })}
    response =  self.lambdaClient.invoke(
        functionName = functionName, input = input )
    try:
      inventory = json.loads(Response.from_dict(response).body)
      return {k:union(v,{'ib_prcode':ib_prcode,'ib_brcode':k}) \
              if k.isdigit() else v for k,v in inventory.items()}
    except:
      return response

  def queryAll(self, functionName = None):
    '''get the whole database'''
    functionName = functionName or self.endpoint.queryAll()
    response =  self.lambdaClient.invoke(
        functionName = functionName, input = {} )
    responseBody = json.loads(Response.from_dict(response).body)
    ### return body
    if 'url' in responseBody:
      inventory =  Requests.getContentFromUrl(responseBody['url'])
      return {k:{k2: union(v2, {'ib_prcode':k,'ib_brcode': k2}) if k2.isdigit() 
    else v2 for k2,v2 in v.items()} if k.isdigit() else v for k,v in inventory.items()}
#       return {k:{k2: union(json.loads(v2), {'ib_prcode':k,'ib_brcode': k2})for k2,v2 in v.items()} for k,v in inventory.items()}
    else :
      logging.error(responseBody)
      return responseBody
  
  def queryBranch(self, branch = '1000', functionName = None):
    '''get the branch database'''
    functionName = functionName or self.endpoint.queryBranch()
    response =  self.lambdaClient.invoke(
        functionName = functionName, input = {'body':json.dumps({'branch':branch})} )
    responseBody = json.loads(Response.from_dict(response).body)
    ### return body
    if 'url' in responseBody:
      inventory = Requests.getContentFromUrl(responseBody['url'])
      return {k:union(v,{'ib_prcode':k,'ib_brcode':branch}) for k,v in inventory.items()}
    else :
      logging.error(responseBody)
      return responseBody

In [ ]:
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from random import randrange
from datetime import datetime
import boto3


## generate dummy data for testing

In [ ]:
%%time
#Dummy Data
numberOfRows = 10
@dataclass_json
@dataclass
class Inventory:
  ib_prcode:str
  ib_brcode:str
  ib_cf_qty:str
  new_ib_vs_stock_cv:str

sampleLargeRandomInput = [ Inventory.from_dict({
    'ib_brcode' : str(randrange(1000,1030,1)),
    'ib_prcode' : str(randrange(10000,100000,1)),
    'ib_cf_qty' : str(randrange(-10,1000,1)),
    'new_ib_vs_stock_cv' : str(randrange(-10,1000,1))
  }).to_dict() for _ in range(numberOfRows)]
sampleLargeRandomInput[0]

CPU times: user 1.97 ms, sys: 0 ns, total: 1.97 ms
Wall time: 1.98 ms


{'ib_prcode': '29753',
 'ib_brcode': '1029',
 'ib_cf_qty': '104',
 'new_ib_vs_stock_cv': '876'}

## create main object

In [ ]:
sdk = InventorySdk(user=USER, pw=PW, branchName = 'dev-manual')

## Upload the batch data using s3

### SampleData

In [ ]:
sampleInput =  [
               {
                  'iprcode': '0000009',
                  'brcode': '1000',
                  'ib_cf_qty': '50',
                  'new_ib_vs_stock_cv': '27'
                },
               {
                  'iprcode': '0000004',
                  'brcode': '1000',
                  'ib_cf_qty': '35',
                  'new_ib_vs_stock_cv': '33'
               },
                {
                  'iprcode': '0000003',
                  'brcode': '1003',
                  'ib_cf_qty': '36',
                  'new_ib_vs_stock_cv': '33'
               }
              ]

def getDf(input_:dict):
  return pd.DataFrame(input_)
  
df = getDf(sampleInput)
df

,iprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv
0,0000009,1000,50,27
1,0000004,1000,35,33
2,0000003,1003,36,33


### uploading to s3

In [ ]:
#export
@add_method(InventorySdk)
def uploadDf(self, df:pd.DataFrame, key:str ='1000', invApi:str = '2y9nzxkuyk')->bin:
  def getPresignedUrl(invApi = invApi, key = key):
    url = f'https://{invApi}.execute-api.ap-southeast-1.amazonaws.com/Prod/presign'
    r:requests.Response = requests.post(url, json = { "key": key } )
    return r.json()
  def dfToByte(df:pd.DataFrame):
    tempIo = BytesIO()
    inputByte = df.to_feather(tempIo)
    return tempIo.getvalue()
  def uploadFile(inputByte:bin, key=key):
    presigned = getPresignedUrl(key=key)
    print('signed url is ')
    printDict(presigned)
    files = {'file': (key , BytesIO(inputByte))}
    r = requests.post(url = presigned['url'], data = presigned['fields'] , files = files)
    return r
  
  ##### main 
  inputByte = dfToByte(df)
  r = uploadFile(inputByte, key = key)
  return r

In [ ]:
#hide
sdk.uploadDf(df, invApi = '2y9nzxkuyk')

signed url is 
url : https://in
fields
 key : 1000
 AWSAccessKeyId : ASIAVX4Z5T
 x-amz-security-token : IQoJb3JpZ2
 policy : eyJleHBpcm
 signature : kri0RzYNXu


<Response [204]>

In [ ]:
#hide
pd.read_csv('sampleData/inventory.csv',index_col=0).reset_index(drop=True)
sdk.uploadDf(df, invApi = '2y9nzxkuyk')

signed url is 
url : https://in
fields
 key : 1000
 AWSAccessKeyId : ASIAVX4Z5T
 x-amz-security-token : IQoJb3JpZ2
 policy : eyJleHBpcm
 signature : 6/TJQ3dZxL


<Response [204]>

### trigger ingestion

In [ ]:
#export
@add_method(InventorySdk)
def ingestData(self, functionName= 'trigger-ingestion-dev-manual', key='1000', dtype='feather'):
  lambda_:Lambda = self.lambdaClient
  result = lambda_.invoke(functionName=functionName, input=Event.getInput({'key':key, 'dtype':dtype}))
  return result

In [ ]:
sdk.ingestData(key = '1000')

{'body': '{"iprcode":{"0":"0000009","1":"0000004","2":"0000003"},"brcode":{"0":"1000","1":"1000","2":"1003"},"ib_cf_qty":{"0":"50","1":"35","2":"36"},"new_ib_vs_stock_cv":{"0":"27","1":"33","2":"33"}}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

### The whole flow

In [ ]:
%%time
key = 'test'
r = sdk.uploadDf(df, key = key)
if r.status_code >= 400: raise Exception(r.json())
sdk.ingestData(key = key)

signed url is 
url : https://in
fields
 key : test
 AWSAccessKeyId : ASIAVX4Z5T
 x-amz-security-token : IQoJb3JpZ2
 policy : eyJleHBpcm
 signature : sjg3Unz1l3
CPU times: user 30.6 ms, sys: 247 µs, total: 30.9 ms
Wall time: 517 ms


{'body': '{"iprcode":{"0":"0000009","1":"0000004","2":"0000003"},"brcode":{"0":"1000","1":"1000","2":"1003"},"ib_cf_qty":{"0":"50","1":"35","2":"36"},"new_ib_vs_stock_cv":{"0":"27","1":"33","2":"33"}}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

## Query a branch

In [ ]:
#export
@add_method(InventorySdk)
def branchQuery(self, brcode:str, cprcodes:List[int]=[])->pd.DataFrame:
  lambda_: Lambda =self.lambdaClient
  payload = Event.getInput({
    'brcode': brcode,
    'cprcodes': [int(i) for i in cprcodes],
    'format' : 'feather'
  })
  rawReturn = lambda_.invoke(functionName=self.endpoint.queryBranch(), input = payload)
  if rawReturn.get('errorMessage'):
    raise Exception(rawReturn)
  parsedReturn = Response.parseBody(rawReturn)
  return pd.read_feather(parsedReturn['url'])
  

In [ ]:
sdk.branchQuery('1000', [47])

,iprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv
0,4,1000,35,33
1,9,1000,95,95
2,12,1000,36,36
3,26,1000,28,28
4,28,1000,9,9
...,...,...,...,...
81546,244801,1000,0,0
81547,244814,1000,0,0
81548,244816,1000,0,0
81549,244817,1000,0,0


## Query All

In [ ]:
#export
@add_method(InventorySdk)
def queryAll2(self, format_ = 'feather'):
  functionName = self.endpoint.queryAll2()
  r =  self.lambdaClient.invoke(
      functionName = functionName, input = Event.getInput({'format': format_ } ))
  if r['statusCode'] > 300 :
    raise Exception(f'error getting database url {r}')
  body = Response.parseBody(r)
  url = body['url']
  print('succesfully get url, returning pandas')
  if format_ == 'feather':
    return pd.read_feather(url).astype(int)
  return pd.read_json(url, orient='split', dtype= int)
  

In [ ]:
%%time
sdk.queryAll2()

succesfully get url, returning pandas
CPU times: user 77.5 ms, sys: 13 ms, total: 90.5 ms
Wall time: 652 ms


,iprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv
0,4,1000,35,33
1,9,1000,95,95
2,12,1000,36,36
3,26,1000,28,28
4,28,1000,9,9
...,...,...,...,...
81547,244814,1000,0,0
81548,244816,1000,0,0
81549,244817,1000,0,0
81550,244818,1000,0,0


In [ ]:
%%time
sdk.queryAll2(format_='json')

succesfully get url, returning pandas
CPU times: user 217 ms, sys: 12.6 ms, total: 230 ms
Wall time: 910 ms


,iprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv
0,4,1000,35,33
1,9,1000,95,95
2,12,1000,36,36
3,26,1000,28,28
4,28,1000,9,9
...,...,...,...,...
81547,244814,1000,0,0
81548,244816,1000,0,0
81549,244817,1000,0,0
81550,244818,1000,0,0


## Query single product 2

In [ ]:
#export 
@add_method(InventorySdk)
def querySingleProduct2(self, iprcode = '1234'):
  res:pd.DataFrame = self.queryAll2()
  if res.empty: raise Exception('database not found')
  return res[res['iprcode']==int(iprcode)]

In [ ]:
sdk.querySingleProduct2(244814)

succesfully get url, returning pandas


,iprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv
81547,244814,1000,0,0


## schema
```
key:str # key is ib_prcode
  ib_cf_qty: int
  new_ib_bs_stock_cv: int
  lastUpdate: float
  ib_brcode: str
  ib_prcode: str
```